In [ ]:
from tensorflow import keras
from kerastuner.tuners import RandomSearch
import tensorflow as tf
from tensorflow.keras.layers import LSTM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns


# Fully Connected Network (FCN) example:
fcn_model = Sequential()
fcn_model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
fcn_model.add(Dense(64, activation='relu'))
fcn_model.add(Dense(1, activation='sigmoid'))

fcn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
fcn_model.fit(X_train, y_train, epochs=20, batch_size=32)


In [ ]:
# CNN example:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))  # Adjust input_shape as needed
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dense(1, activation='sigmoid'))

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32)


def build_model(hp):
    model = keras.Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=128, step=32),
                    activation='relu', input_dim=X_train.shape[1]))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Float('lr', min_value=1e-4, max_value=1e-2, sampling='LOG')), 
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=5, executions_per_trial=3, directory='my_dir')
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
best_model = tuner.get_best_models(1)[0]

def physics_informed_loss(y_true, y_pred):
    # This is just a placeholder; you'd need to define how to calculate physics-informed loss
    physics_loss = tf.reduce_mean(tf.abs(y_true - y_pred))  # Simple difference (customize as per application)
    return physics_loss + tf.keras.losses.binary_crossentropy(y_true, y_pred)

# Compile the model with custom loss function
model.compile(optimizer='adam', loss=physics_informed_loss, metrics=['accuracy'])


In [ ]:
hybrid_model = Sequential()
hybrid_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
hybrid_model.add(MaxPooling2D(pool_size=(2, 2)))
hybrid_model.add(Flatten())
hybrid_model.add(LSTM(128, activation='relu', return_sequences=True))
hybrid_model.add(Dense(1, activation='sigmoid'))

hybrid_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
hybrid_model.fit(X_train, y_train, epochs=10, batch_size=32)

# Experiment with different activations and visualizations
model.add(Dense(128, activation='tanh'))  # Experiment with tanh instead of ReLU



In [ ]:
# For each model (Random Forest, SVM, Gradient Boosting, Neural Networks, etc.), calculate the metrics:

# Example: Evaluate Random Forest
y_pred_rf = rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)
rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)

# Example: Evaluate Deep Learning Model (e.g., CNN)
y_pred_dl = model.predict(X_test)
accuracy_dl = accuracy_score(y_test, y_pred_dl)
precision_dl = precision_score(y_test, y_pred_dl)
recall_dl = recall_score(y_test, y_pred_dl)
f1_dl = f1_score(y_test, y_pred_dl)
rmse_dl = mean_squared_error(y_test, y_pred_dl, squared=False)

# Print or log the results for each model:
print("Random Forest Metrics:")
print(f"Accuracy: {accuracy_rf:.4f}, Precision: {precision_rf:.4f}, Recall: {recall_rf:.4f}, F1: {f1_rf:.4f}, RMSE: {rmse_rf:.4f}")
print("Deep Learning Model Metrics:")
print(f"Accuracy: {accuracy_dl:.4f}, Precision: {precision_dl:.4f}, Recall: {recall_dl:.4f}, F1: {f1_dl:.4f}, RMSE: {rmse_dl:.4f}")

# You can also put the results in a DataFrame for comparison:
metrics_df = pd.DataFrame({
    'Model': ['Random Forest', 'Deep Learning'],
    'Accuracy': [accuracy_rf, accuracy_dl],
    'Precision': [precision_rf, precision_dl],
    'Recall': [recall_rf, recall_dl],
    'F1 Score': [f1_rf, f1_dl],
    'RMSE': [rmse_rf, rmse_dl]
})
print(metrics_df)


In [ ]:
# Assuming you have a separate validation dataset or an out-of-distribution dataset:
X_out_of_distribution, y_out_of_distribution = load_out_of_distribution_data()

# Evaluate Random Forest on unseen data:
y_pred_ood_rf = rf.predict(X_out_of_distribution)
accuracy_ood_rf = accuracy_score(y_out_of_distribution, y_pred_ood_rf)

# Evaluate Deep Learning Model on unseen data:
y_pred_ood_dl = model.predict(X_out_of_distribution)
accuracy_ood_dl = accuracy_score(y_out_of_distribution, y_pred_ood_dl)

print(f"Random Forest Accuracy on Out-of-Distribution Data: {accuracy_ood_rf:.4f}")
print(f"Deep Learning Model Accuracy on Out-of-Distribution Data: {accuracy_ood_dl:.4f}")

# Overfitting vs. Underfitting Discussion
# Overfitting: If the model performs much better on the training set than on the test/out-of-distribution set, it may be overfitting.
# Underfitting: If the model performs poorly on both the training and test/out-of-distribution sets, it may be underfitting.

# Plot loss/accuracy curves to check for overfitting
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Model Performance Over Epochs")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

from sklearn.metrics import confusion_matrix, roc_curve, auc
import seaborn as sns

# Confusion Matrix for Random Forest
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(conf_matrix_rf, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Random Forest')
plt.show()

# ROC Curve for Deep Learning Model
fpr, tpr, _ = roc_curve(y_test, y_pred_dl)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve - Deep Learning Model')
plt.legend(loc="lower right")
plt.show()

# Loss vs Epochs plot (for deep learning models)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# If spatial/temporal error maps are needed, visualize errors across geospatial regions or over time (based on dataset).